In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim

import dlc_practical_prologue as prologue

In [2]:
N = 1000 #nb of pairs

#generate pairs
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(N)

In [3]:
#normalize the input
train_input/=255
test_input/=255

### 2 Convolution layers, stride = 2

Results: lr = 0.001, train error: 5%, test error: 17%

In [6]:
class Net(nn.Module):
    def __init__(self, nb_hidden):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32,64, kernel_size=3)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2, stride=2)) #6x6
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2)) #conv : 4x4, maxpool : 2x2
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x

In [70]:
def train_model(model, train_input, train_target, mini_batch_size, lr, nb_epoch):
    #criterion = nn.MSELoss()
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr)
    
    for e in range(nb_epoch):
        sum_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            optimizer.step()
            
            sum_loss = sum_loss + loss.item()
        #print(e, sum_loss)

In [23]:
def compute_nb_errors_targets(model, input, target):
    nb_errors = 0
    output = model(input)
    _, predicted_target = output.max(1) #max probabilities of target
    #print(predicted_target)
    
    for b in range(1000):
        if target[b,int(predicted_target[b])] <= 0:
            nb_errors = nb_errors + 1
            
    return nb_errors

In [9]:
#train_target[1000,1]

new_train_target = torch.empty(1000,2)
new_test_target = torch.empty(1000,2)
for i in range(1000):
    if train_target[i] == 1 :
        new_train_target[i,0] = 0
        new_train_target[i,1] = 1
        
    else:
        new_train_target[i,0] = 1
        new_train_target[i,1] = 0
        
    if test_target[i] == 1:
        new_test_target[i,0] = 0
        new_test_target[i,1] = 1
        
    else:
        new_test_target[i,0] = 1
        new_test_target[i,1] = 0

#### Choice of parameters

In [73]:
#choose nb_epoch --> nb_epoch = 25

mini_batch_size = 100
lr = 0.001

#for nb_epoch in [25, 50, 75]: --> nb_epoch = 25: the other overfit
for nb_epoch in [10, 15, 20, 25]:
    
    model = Net(64)
    train_model(model, train_input, new_train_target, mini_batch_size, lr, nb_epoch)
    print("i =", nb_epoch)

    nb_train_errors = compute_nb_errors_targets(model, train_input, new_train_target)
    print('train error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_train_errors) / train_input.size(0),
                                                      nb_train_errors, train_input.size(0)))
    nb_test_errors = compute_nb_errors_targets(model, test_input, new_test_target)
    print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                    nb_test_errors, test_input.size(0)))

i = 9
train error Net 19.30% 193/1000
test error Net 21.40% 214/1000
i = 9
train error Net 13.30% 133/1000
test error Net 19.30% 193/1000
i = 9
train error Net 6.30% 63/1000
test error Net 18.30% 183/1000
i = 9
train error Net 6.90% 69/1000
test error Net 17.30% 173/1000


In [28]:
#Choice of learning rate: lr = 0.001 : train error: 7%, test error: 18%
#for lr in [0.001,0.005, 0.01, 0.05, 0.1, 0.5]:
for lr in [0.0005, 0.001, 0.005]:
    model = Net(64)
    mini_batch_size = 100
    nb_epoch = 25

    train_model(model, train_input, new_train_target, mini_batch_size, lr, nb_epoch)
    print("LR =", lr)
    
    nb_train_errors = compute_nb_errors_targets(model, train_input, new_train_target)
    print('train error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_train_errors) / train_input.size(0),
                                                      nb_train_errors, train_input.size(0)))
    nb_test_errors = compute_nb_errors_targets(model, test_input, new_test_target)
    print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                    nb_test_errors, test_input.size(0)))

LR = 0.0005
train error Net 10.80% 108/1000
test error Net 19.40% 194/1000
LR = 0.001
train error Net 7.20% 72/1000
test error Net 18.60% 186/1000
LR = 0.005
train error Net 0.00% 0/1000
test error Net 17.40% 174/1000


#### Final test

In [30]:
mini_batch_size = 100
nb_epoch = 25
lr = 0.001

for i in range(10):
    model = Net(64)
    train_model(model, train_input, new_train_target, mini_batch_size, lr, nb_epoch)
    print("i =", i)

    nb_train_errors = compute_nb_errors_targets(model, train_input, new_train_target)
    print('train error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_train_errors) / train_input.size(0),
                                                      nb_train_errors, train_input.size(0)))
    nb_test_errors = compute_nb_errors_targets(model, test_input, new_test_target)
    print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                    nb_test_errors, test_input.size(0)))

i = 0
train error Net 5.00% 50/1000
test error Net 18.20% 182/1000
i = 1
train error Net 5.10% 51/1000
test error Net 17.50% 175/1000
i = 2
train error Net 5.70% 57/1000
test error Net 19.30% 193/1000
i = 3
train error Net 5.20% 52/1000
test error Net 18.70% 187/1000
i = 4
train error Net 5.60% 56/1000
test error Net 17.10% 171/1000
i = 5
train error Net 6.90% 69/1000
test error Net 19.00% 190/1000
i = 6
train error Net 2.80% 28/1000
test error Net 17.10% 171/1000
i = 7
train error Net 3.10% 31/1000
test error Net 16.40% 164/1000
i = 8
train error Net 6.80% 68/1000
test error Net 17.00% 170/1000
i = 9
train error Net 5.10% 51/1000
test error Net 17.20% 172/1000
